In [9]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from price_impact import regression_path
from price_impact import tickers as spots
from price_impact.modelling.backtest_analysis import (BacktestResults, BacktestSettings)
from price_impact.modelling.Futures import FutureContract, Spot

In [10]:
results_list = []

backtest_settings = BacktestSettings(
    train_len=6,
    test_len=3,
    bin_freq='10s',
    forecast_horizon_s=10,
    smoothing_str='sma20',
    decay=3600

)


def get_stat_dict(spot):
    backtest_results = BacktestResults(spot, backtest_settings)
    _, _, _ = backtest_results.create_and_save_plots()
    stat_dict = backtest_results.statistics_dict
    stat_dict['spot'] = spot
    return stat_dict


with ThreadPoolExecutor(10) as executor:
    results_list = executor.map(get_stat_dict, spots)

results_list = [res for res in results_list if res is not None]

### Above code will have generated 3d plots and $R^2$ plots in `price_impact/db/regression/10s/10/results/summary`

In [17]:
df = pd.DataFrame(results_list).sort_values("pct_curr_best_is_next_best")
df['spot'] = df['spot'].apply(lambda x: Spot(x))
df['asset_class'] = df['spot'].apply(lambda x: x.asset_class)

In [18]:

df['active_months']= df['spot'].apply(lambda x: len(FutureContract.from_spot(x).active_months))
df['active_hours']= df['spot'].apply(lambda x: (x.trading_end_time - x.trading_start_time).seconds/60/60 )
df['description']= df['spot'].apply(lambda x: x.description)
df = df.sort_values(by=['asset_class','avg_tr_fr_tr']).astype({'best':'float'})

# print(df[['avg_tr_fr_tr','avg_te_fr_tr']].mean())

df = df.sort_values(by=['avg_tr_fr_tr']).astype({'best': 'float'}).reset_index(drop=True)
df = df[['spot', 'description', 'avg_tr_fr_tr', 'avg_te_fr_tr', 'test_train_ratio',
         'avg_max_test_regret', 'best', 'pct_best', 'pct_curr_best_is_next_best']]


df = df.rename(columns={'avg_tr_fr_tr': r'Train $R^2$', 'avg_te_fr_tr': r'Test $R^2$', 'avg_max_test_regret': 'Test Regret',
                        'pct_curr_best_is_next_best': 'Stability',
                        'pct_best': r'$\hat{\delta}$ Freq', 'best': r'$\hat{\delta}$',
                         'description': 'Description',
                        'test_train_ratio': 'TT Ratio'})
df

,spot,Description,Train $R^2$,Test $R^2$,TT Ratio,Test Regret,$\hat{\delta}$,$\hat{\delta}$ Freq,Stability
0,ES,S&P 500 E-Mini Futures,0.402570,0.388908,0.972031,0.008141,0.10,0.418605,0.523810
1,CL,Light Crude Oil Futures NYMEX,0.529470,0.519943,0.984796,0.006566,0.35,0.209302,0.428571
2,NQ,NASDAQ 100 E-Mini Futures,0.540839,0.526523,0.974768,0.003348,0.65,0.302326,0.547619
3,GC,Gold Futures COMEX,0.542381,0.535821,0.989519,0.005345,0.45,0.209302,0.404762
4,GL,Long Gilt Futures,0.594837,0.593746,1.006358,0.001176,0.10,0.837209,0.833333
5,CN,Corn Futures,0.755070,0.754856,1.002523,0.000192,0.10,0.692308,0.750000
6,TU,US 2-Year T-Note Futures,0.775369,0.759069,0.980480,0.000159,0.10,1.000000,1.000000
